In [1]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import datetime
import glob

In [2]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

In [3]:
start_year = 2011
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['PREC_ACC_NC'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['PREC_ACC_NC'].dims[1:], coords={'south_north': ds['PREC_ACC_NC'].coords['south_north'], 'west_east': ds['PREC_ACC_NC'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s.PREC_ACC_NC.values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'p': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)
                        

2024-10-06 17:11:25
2024-10-06 17:26:00
2024-10-06 17:39:10
2024-10-06 17:53:40
2024-10-06 18:08:46
2024-10-06 18:21:52
2024-10-06 18:36:14
2024-10-06 18:51:43
2024-10-06 19:05:58
2024-10-06 19:19:42
2024-10-06 19:34:01
2024-10-06 19:47:51
2024-10-06 20:00:59
2024-10-06 20:14:19
2024-10-06 20:28:32
2024-10-06 20:42:36
2024-10-06 20:56:37
2024-10-06 21:08:54
2024-10-06 21:23:11
2024-10-06 21:37:37
2024-10-06 21:53:26
2024-10-06 22:08:09
2024-10-06 22:24:49
2024-10-06 22:33:21


In [2]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 2011
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.T2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['T2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['T2'].dims[1:], coords={'south_north': ds['T2'].coords['south_north'], 'west_east': ds['T2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['T2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'t2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'T2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-02 20:41:50
2024-11-02 20:55:49
2024-11-02 21:09:13
2024-11-02 21:22:55
2024-11-02 21:36:38
2024-11-02 21:49:30
2024-11-02 22:03:22
2024-11-02 22:16:38
2024-11-02 22:30:25
2024-11-02 22:45:03
2024-11-02 22:59:00
2024-11-02 23:13:48
2024-11-02 23:26:47
2024-11-02 23:40:29
2024-11-02 23:53:13
2024-11-03 00:06:01
2024-11-03 00:19:58
2024-11-03 00:32:37
2024-11-03 00:47:13
2024-11-03 01:00:28
2024-11-03 01:14:04
2024-11-03 01:28:14
2024-11-03 01:42:23
2024-11-03 01:57:21


In [3]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 2011
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)

    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:

            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  
            else:
                num_days = 28  

        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.TD2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)

            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['TD2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['TD2'].dims[1:], coords={'south_north': ds['TD2'].coords['south_north'], 'west_east': ds['TD2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['TD2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'td2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'TD2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-03 01:10:57
2024-11-03 01:25:25
2024-11-03 01:39:24
2024-11-03 01:54:58
2024-11-03 02:09:07
2024-11-03 02:22:59
2024-11-03 02:36:44
2024-11-03 02:49:57
2024-11-03 03:03:25
2024-11-03 03:16:46
2024-11-03 03:29:57
2024-11-03 03:43:08
2024-11-03 03:56:22
2024-11-03 04:10:51
2024-11-03 04:24:06
2024-11-03 04:38:11
2024-11-03 04:51:34
2024-11-03 05:03:48
2024-11-03 05:17:52
2024-11-03 05:31:04
2024-11-03 05:44:08
2024-11-03 05:56:43
2024-11-03 06:09:29
2024-11-03 06:22:11


In [2]:
input_folder_t = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern_t = 'T2.wrf2d_d01_????-??-??.nc'
file_pattern_dt = 'TD2.wrf2d_d01_????-??-??.nc'
output = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA_dailydata/'


folder_names = ['U-58', 
     # 'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
     # 'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
     # 'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
     # 'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]
for folder in folder_names:
    print(datetime.datetime.now().time())
    full_path_t = os.path.join(base_path, folder, file_pattern_t)
    full_path_dt = os.path.join(base_path, folder, file_pattern_dt)
    all_files_t = glob.glob(full_path_t)
    all_files_dt = glob.glob(full_path_dt)
    #####
    summer_files_t = [f for f in all_files_t if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    summer_files_dt = [f for f in all_files_dt if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    
    
    ds_t = xr.open_mfdataset(summer_files_t)
    ds_t = ds_t.sel(time=ds_t['time'].dt.month.isin([6, 7, 8]))
    ds_dt = xr.open_mfdataset(summer_files_dt)
    ds_dt = ds_dt.sel(time=ds_dt['time'].dt.month.isin([6, 7, 8]))    
    

    grouped_t = ds_t.groupby('time.year').groups
    grouped_dt = ds_dt.groupby('time.year').groups

    for year, year_indices_t in grouped_t.items():
        year_indices_t = grouped_t[year]
        year_indices_dt = grouped_dt[year]

        ds_year_t = ds_t.isel(time=year_indices_t)
        ds_year_dt = ds_dt.isel(time=year_indices_dt)

        monthly_groups_t = ds_year_t.groupby('time.month').groups
        monthly_groups_dt = ds_year_dt.groupby('time.month').groups

        for month, month_indices_t in monthly_groups_t.items():
            month_indices_t = monthly_groups_t[month]
            month_indices_dt = monthly_groups_dt[month]

            ds_month_t = ds_year_t.isel(time=month_indices_t)
            ds_month_dt = ds_year_dt.isel(time=month_indices_dt)

            daily_groups_t = ds_month_t.groupby('time.day').groups
            daily_groups_dt = ds_month_dt.groupby('time.day').groups

            # 循环遍历每日的数据
            for day, day_indices_t in daily_groups_t.items():
                day_indices_t = daily_groups_t[day]
                day_indices_dt = daily_groups_dt[day]

                ds_day_t = ds_month_t.isel(time=day_indices_t)
                ds_day_dt = ds_month_dt.isel(time=day_indices_dt)

                filename_t = f'{folder}_mt_{year}_{month:02d}_{day:02d}.nc'
                filename_dt = f'{folder}_mdt_{year}_{month:02d}_{day:02d}.nc'

                # 导出每日数据为 NetCDF 文件
                ds_day_t.to_netcdf(output+filename_t)
                ds_day_dt.to_netcdf(output+filename_dt)

00:53:18.435351
